In [ ]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained(
    model_path_or_repo_id="/mnt/DATA/THESIS/playground/models/mistral-7b-instruct-v0.1.Q5_K_M.gguf",
    model_file="./models/mistral-7b-instruct-v0.1.Q5_K_M.gguf",
    model_type="mistral", 
    gpu_layers=30)


In [ ]:
import os
import re

# Define a function to preprocess text
def preprocess_text(text):
    
    text = text.lower()
    # Split the text into sentences
    sentences = re.split(r'[.!?]', text)
    
    for i,sentence in enumerate(sentences):
        sentence = sentence.split("|")[-1]
        sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)
        sentences[i] = sentence.split("|")[-1]
    
    return sentences

# Load the dataset from a folder
dataset_folder = "documents/topic_model_dataset"
sentences = []

# Iterate over files in the dataset folder
for filename in os.listdir(dataset_folder):
    if filename.endswith(".txt"):
        with open(os.path.join(dataset_folder, filename), "r", encoding="utf-8") as file:
            text = file.read()
            sentences.extend(preprocess_text(text))



print(f'length of documents: {len(sentences)}')
print(f"first sentences:\n", "\n".join(sentences[:5]))

In [ ]:
from pipeline.embeddings.basic_embeddings import Embedding

embed_model = Embedding()

embeddings = []

for sentence in sentences:
    embeddings.append(embed_model.encode({"text":sentence, "source":"test"}))

print(embeddings[:5])      

In [ ]:
print(embeddings[2])

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import umap


# embeddings = umap.UMAP(n_neighbors=15, 
#                     n_components=10, 
#                     min_dist=0.0, 
#                     metric='cosine',
#                    ).fit_transform(embeddings)


scores = []
for k in range(5,6):
    kmeans = KMeans(n_clusters=k, random_state=42).fit(embeddings)
    score = silhouette_score(embeddings, kmeans.labels_)
    scores.append(score)

best_k = scores.index(max(scores)) + 5 
documents = KMeans(n_clusters=best_k, random_state=42).fit(embeddings)

In [ ]:
best_k

In [ ]:
documents.__dict__

In [ ]:
from pipeline.embeddings.basic_embeddings import Embedding

embed_model = Embedding()

# select the topic words and decode them
topic_words = []
for i in range(best_k):
    topic_words.append([embed_model.decode(word) for word in embeddings[documents.labels_ == i].mean(axis=0).argsort()[-10:][::-1]])
print(topic_words)


In [ ]:
# k * vocab
X_per_cluster = model.transform(documents)
# D * vocab
X_origin = self.vectorizer_model.transform(origin_documents)

if self.word_select_method == 'tfidf_idfi':
    socres = TFIDF_IDFi(X_per_cluster, X_origin, documents).socre()
elif self.word_select_method == 'tfidf_tfi':
    socres = TFIDF_TFi(X_per_cluster, X_origin, all_documents).socre()
elif self.word_select_method == 'tfi':
    socres = TFi(X_per_cluster).socre()
elif self.word_select_method == 'tfidfi':
    socres = TFIDFi(X_per_cluster).socre()

In [ ]:
import pipeline.pseudo_oracle.oracle.SentenceClassiier as p
p

In [ ]:
1 3 4 5 6 7 8 9 10 13 14 15 35 37 39 41 42 47 50 53 62 82 83 118 138